<a href="https://colab.research.google.com/github/bhatiap70077/Natural-Language-Processing/blob/main/Week11_word_vactors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk 
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import re
from scipy.spatial import distance
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Read the data
a=[]
b=[]
with open('airline_review_training_data.csv','r') as f:
    l=f.readlines()
    for j in l:
        a.append(j.split('\t')[0])
        b.append(j.split('\t')[0])

In [ ]:
# convert to pandas 
d={'label':a,'document':b}
df=pd.DataFrame(d)
df.head()

,label,document
0,"text,airline_sentiment\n","text,airline_sentiment\n"
1,"""@USAirways BF has been stuck in CLT all day. ...","""@USAirways BF has been stuck in CLT all day. ..."
2,@united DM sent. This lack if customer servic...,@united DM sent. This lack if customer servic...
3,"@JetBlue thank you for the information.,1\n","@JetBlue thank you for the information.,1\n"
4,@AmericanAir I DMed you my AA &amp; phone #s &...,@AmericanAir I DMed you my AA &amp; phone #s &...


In [ ]:
# taking a sample of data
full_data = df

In [ ]:
# get stop words
# from nltk.corpus import stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# lower case

sample_data1 = (
    # start with training data
    full_data
    # convert to lower case
    .assign(clean_document= lambda x:[text.lower() for text in x.document]) 
    # remove punctuation
    .assign(clean_document= lambda x: [re.sub("[^\w\s]","",text) for text in x.clean_document])
     # remove numbers
    .assign(clean_document= lambda x: [re.sub("\d","",text) for text in x.clean_document])
    # remove leading & trainling spaces 
    .assign(clean_document = lambda x: [text.strip() for text in x.clean_document])
    # remove all the '\n' values with space
    .assign(clean_document= lambda x: [re.sub(r"\n","",text) for text in x.clean_document] )
    # word tokenize
    .assign(word_token= lambda x: [nltk.word_tokenize(text) for text in x.clean_document] )
    # # remove stop words
    .assign(word_token= lambda x: [list(set(list_of_words).difference(stop_words)) for list_of_words in x.word_token])
    # Lemmatising the words
    # .assign(word_token= lambda x: [[lem.lemmatize(a_word) for a_word in list_of_words] for list_of_words in x.word_token ])
    #  # input to sklearn
    .assign(document_to_sklearn= lambda x: [" ".join(map(str,list_of_words)) for list_of_words in x.word_token ])
)

In [ ]:
sample_data1

,label,document,clean_document,word_token,document_to_sklearn
0,"text,airline_sentiment\n","text,airline_sentiment\n",textairline_sentiment,[textairline_sentiment],textairline_sentiment
1,"""@USAirways BF has been stuck in CLT all day. ...","""@USAirways BF has been stuck in CLT all day. ...",usairways bf has been stuck in clt all day is ...,"[weather, free, day, want, clt, lounge, amp, t...",weather free day want clt lounge amp tequila s...
2,@united DM sent. This lack if customer servic...,@united DM sent. This lack if customer servic...,united dm sent this lack if customer service ...,"[sent, getting, ridiculous, dm, service, custo...",sent getting ridiculous dm service customer la...
3,"@JetBlue thank you for the information.,1\n","@JetBlue thank you for the information.,1\n",jetblue thank you for the information,"[thank, information, jetblue]",thank information jetblue
4,@AmericanAir I DMed you my AA &amp; phone #s &...,@AmericanAir I DMed you my AA &amp; phone #s &...,americanair i dmed you my aa amp phone s amp y...,"[someone, didnt, phone, response, dmed, point,...",someone didnt phone response dmed point aa amp...
...,...,...,...,...,...
8166,@JetBlue Apparently the pilot had made some an...,@JetBlue Apparently the pilot had made some an...,jetblue apparently the pilot had made some ann...,"[fuel, quickly, made, trouble, losing, announc...",fuel quickly made trouble losing announcement ...
8167,@united customer service sucks! They hang up ...,@united customer service sucks! They hang up ...,united customer service sucks they hang up af...,"[sucks, minutes, hang, service, waiting, hour,...",sucks minutes hang service waiting hour custom...
8168,@USAirways I paid for my seat. I expect to be...,@USAirways I paid for my seat. I expect to be...,usairways i paid for my seat i expect to be a...,"[usairways, paid, expect, use, seat, able, full]",usairways paid expect use seat able full
8169,"""@united so, not only were you Late Flight, yo...","""@united so, not only were you Late Flight, yo...",united so not only were you late flight you br...,"[paying, one, new, httptconyvvxd, flight, hand...",paying one new httptconyvvxd flight handle lug...


In [ ]:
from collections import defaultdict

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()
    for text in sentences:
        # preprocessing (use tokenizer instead)
        text = text.lower().split()
        # iterate over sentences
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)  # add to vocab
            next_token = text[i+1 : i+1+window_size]
            for t in next_token:
                key = tuple( sorted([t, token]) )
                d[key] += 1
    
    # formulate the dictionary into dataframe
    vocab = sorted(vocab) # sort vocab
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)
    for key, value in d.items():
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

In [ ]:
text = sample_data1['document_to_sklearn']
data = co_occurrence(text, 10)
data

,_,_austrian,_emmaclifford,_exact_,_robprice,aa,aaadvantage,aacom,aadelay,aadfw,...,zero,zigzagging,zippers,zkatcher,zone,zones,zoom,zrh,zukes,zurich
_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_austrian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_emmaclifford,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_exact_,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_robprice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zones,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoom,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zrh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zukes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.decomposition import KernelPCA
pca = KernelPCA(10)
pca_d = pca.fit_transform(data)
data1 = pd.DataFrame(pca_d,index=data.index,columns=['pca1','pca2','pca3','pca4','pc5','pca6','pca7','pca8','pca9','pca10'])

In [ ]:
data1

,pca1,pca2,pca3,pca4,pc5,pca6,pca7,pca8,pca9,pca10
_,-3.968763,-0.839943,0.407878,-0.281015,0.019391,-0.626215,0.204955,-0.198497,-0.095116,0.085211
_austrian,-4.403775,-0.314621,0.784282,0.035103,0.076327,-0.117132,0.119782,0.044040,-0.426970,-0.059972
_emmaclifford,-4.554394,-0.349692,0.516106,-0.482520,0.007414,0.255033,-0.007594,-0.594344,0.191316,0.046971
_exact_,-4.318212,-0.388031,0.663436,-0.593281,0.109333,0.646713,-0.404069,0.643085,0.320121,-0.005716
_robprice,-4.341621,-0.348223,0.651354,-0.588155,0.093061,0.527395,-0.373372,0.488706,0.434151,-0.359417
...,...,...,...,...,...,...,...,...,...,...
zones,-4.373678,-0.896913,1.094931,-0.366173,0.076305,0.290736,-0.062979,-0.205657,-0.013614,-0.152852
zoom,-4.631562,-0.338046,0.462222,-0.471463,-0.002772,0.339129,0.020800,-0.653308,0.207465,-0.071850
zrh,-4.304755,-1.268616,0.220276,-0.416511,0.033991,-0.251446,-0.117852,-0.087629,-0.022746,-0.562179
zukes,-4.612806,-0.208577,0.558980,-0.006412,0.059558,-0.109157,0.080761,0.004024,-0.465222,-0.296197


In [ ]:
import plotly.express as px

In [ ]:
data1.index

Index(['_', '_austrian', '_emmaclifford', '_exact_', '_robprice', 'aa',
       'aaadvantage', 'aacom', 'aadelay', 'aadfw',
       ...
       'zero', 'zigzagging', 'zippers', 'zkatcher', 'zone', 'zones', 'zoom',
       'zrh', 'zukes', 'zurich'],
      dtype='object', length=9787)

In [ ]:
fig = px.scatter( x=data1["pca1"], y=data1["pca2"],color=data.index)
fig.show()

In [ ]:
# build a function, that , given a word , returns the closest word....

In [ ]:
def find_match(input:str)->dict:
    
    # Creating dictionaries for distance calculations
    dis_euclidean = {}
    dis_cosine = {}
    
    # getting the dimensions for word
    arr1 = np.array(data1.loc[input])
    
    # looping through our PCA data  and measuring distance 
    for item in data1.index:
        arr2 = np.array(data1.loc[item])
        dis_euclidean[item] = distance.euclidean(arr1, arr2)
        dis_cosine[item] = distance.cosine(arr1, arr2)
    
    # Sorting and creating dataframe
    euclidean = pd.DataFrame.from_dict(dis_euclidean, orient='index',columns=['euclidean']).sort_values(by="euclidean")
    cosine = pd.DataFrame.from_dict(dis_cosine, orient='index',columns=['cosine']).sort_values(by="cosine")
    
    # Giving Top 3 from both distances
    result = {}
    result['Euclidean'] = euclidean.head(6).tail(5).to_dict()["euclidean"]
    result['Cosine'] = cosine.head(6).tail(5).to_dict()["cosine"]
    
    return result

In [ ]:
a=find_match("sorry")
a

{'Cosine': {'expect': 0.011134313752552139,
  'nothing': 0.012208679190610972,
  'travel': 0.006602201175571865,
  'use': 0.008799602011364871,
  'working': 0.009222388135394888},
 'Euclidean': {'fix': 6.679187230447388,
  'money': 6.8311722943939674,
  'problem': 6.5937815157395825,
  'use': 6.009513511415778,
  'working': 6.195589006685693}}

In [ ]:
b=find_match("like")
b

{'Cosine': {'also': 0.005879251089885518,
  'good': 0.00655971045858883,
  'many': 0.006971906514186554,
  'nice': 0.005217662361293285,
  'youre': 0.006771742495858546},
 'Euclidean': {'airline': 24.89885632419231,
  'fly': 28.619496503324388,
  'guys': 25.338571617218438,
  'never': 27.220340116729066,
  'would': 30.69195074889203}}

In [ ]:

c=find_match("good")
c

{'Cosine': {'fly': 0.00727568535545664,
  'like': 0.00655971045858883,
  'right': 0.0036834485427728048,
  'yes': 0.009161880465851535,
  'youre': 0.00890056220183455},
 'Euclidean': {'first': 16.48363861513115,
  'flying': 17.507895183424935,
  'getting': 17.02430464556383,
  'go': 17.569885692058552,
  'know': 16.634014085840985}}

In [ ]:
d=find_match("hour")
d

{'Cosine': {'fail': 0.008614454992234788,
  'min': 0.003604200355095011,
  'mins': 0.004086257703244334,
  'still': 0.01374776248899412,
  'three': 0.013160434967182555},
 'Euclidean': {'amp': 56.64367761068362,
  'gate': 46.09287138757103,
  'plane': 67.51910687845597,
  'still': 37.28874620946718,
  'waiting': 65.78264093092682}}

In [ ]:
e=find_match("please")
e

{'Cosine': {'cant': 0.009640952236401046,
  'day': 0.011699980146531397,
  'help': 0.012176218353476043,
  'travel': 0.012752077858427024,
  'w': 0.012854610860600468},
 'Euclidean': {'back': 38.65147071731982,
  'dont': 34.81928648525507,
  'need': 35.94398179588221,
  'one': 34.99885410936697,
  'trying': 33.01851126989474}}

In [ ]:
e=find_match("flight")
e

{'Cosine': {'flights': 0.06528069848046558,
  'long': 0.08387031459778527,
  'options': 0.07154404995859986,
  'reason': 0.08364378587344279,
  'shit': 0.08886641335927037},
 'Euclidean': {'americanair': 1016.0958217931058,
  'get': 895.8547598995411,
  'help': 1013.0358067726916,
  'hours': 1041.7524614302538,
  'thanks': 1024.407768295388}}

In [ ]:
e=find_match("americanair")
e

{'Cosine': {'jetblue': 0.10942854945941827,
  'southwestair': 0.040460331961327034,
  'united': 0.10803095267746732,
  'usairways': 0.0732193024987442,
  'virginamerica': 0.08503179205706612},
 'Euclidean': {'get': 540.6278014897619,
  'jetblue': 516.2862362945777,
  'southwestair': 346.4940678441729,
  'united': 406.03052601551155,
  'usairways': 337.12965079843707}}

In [ ]:
e=find_match("day")
e

{'Cosine': {'cant': 0.00582652919123916,
  'dont': 0.0063401897435703125,
  'finally': 0.006950250164621807,
  'one': 0.0070687400085061025,
  'w': 0.0074070884243790935},
 'Euclidean': {'agent': 20.862745442739516,
  'airport': 22.28894816918187,
  'even': 21.527636388403547,
  'going': 15.125674562194769,
  'way': 21.502947828172204}}